In [1]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [2]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
headers = {"X-Naver-Client-Id": "lJrKhZKFXCGXSoYzh7f7",
           "X-Naver-Client-Secret":"YYPQvTkBxR"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [3]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        {"name": "PC", "param": [ "50000089"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "광학기기/용품", "param": [ "50000207"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["디지털/가전"], axis=1, inplace=True)
df_all_a

,휴대폰액세서리,PC,노트북,PC부품,노트북액세서리,소프트웨어,주변기기,게임기/타이틀,계절가전,광학기기/용품,...,음향가전,이미용가전,자동차기기,주방가전,카메라/캠코더용품,태블릿PC액세서리,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,1.43483,0.09399,0.30502,0.32940,0.09436,0.04875,0.82239,0.32281,1.11915,0.01257,...,1.02346,0.38094,0.20504,1.84242,0.17984,0.33816,0.01649,PC+MO,전체,전체
2021-05-11,1.37692,0.08884,0.27591,0.31580,0.09136,0.04819,0.77576,0.27776,1.10075,0.01224,...,0.94563,0.35392,0.19436,1.64319,0.15274,0.33506,0.01514,PC+MO,전체,전체
2021-05-12,1.41037,0.08398,0.26763,0.31932,0.08920,0.04674,0.76831,0.25925,1.16766,0.01142,...,0.94872,0.34595,0.18748,1.60637,0.14968,0.32561,0.01485,PC+MO,전체,전체
2021-05-13,1.31968,0.07725,0.23890,0.30471,0.08314,0.04337,0.71071,0.24180,1.55491,0.01153,...,0.90633,0.34923,0.18634,1.55192,0.13434,0.30578,0.01349,PC+MO,전체,전체
2021-05-14,1.24008,0.07210,0.23315,0.28996,0.08032,0.03804,0.64772,0.23084,1.85131,0.01067,...,0.88298,0.33206,0.17345,1.42553,0.12583,0.29567,0.01213,PC+MO,전체,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,0.83805,0.02572,0.09213,0.13828,0.04174,0.01584,0.27031,0.15791,1.22680,0.01032,...,0.42594,0.20780,0.09933,0.89011,0.07228,0.11463,0.01360,MO,전체,전체
2023-06-30,0.74230,0.02430,0.08913,0.13115,0.03927,0.01529,0.26264,0.15347,1.04023,0.00919,...,0.40686,0.19076,0.09665,0.80366,0.06939,0.10718,0.01256,MO,전체,전체
2023-07-01,0.87701,0.02605,0.09717,0.13168,0.04090,0.01602,0.26944,0.19218,1.22208,0.01106,...,0.46611,0.22346,0.10088,0.92237,0.08707,0.12202,0.01329,MO,전체,전체


In [4]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        {"name": "PC", "param": [ "50000089"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "광학기기/용품", "param": [ "50000207"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"f",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["디지털/가전"], axis=1, inplace=True)
df_all_b

,휴대폰액세서리,PC,노트북,PC부품,노트북액세서리,소프트웨어,주변기기,게임기/타이틀,계절가전,광학기기/용품,...,음향가전,이미용가전,자동차기기,주방가전,카메라/캠코더용품,태블릿PC액세서리,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,2.04403,0.06155,0.29513,0.08021,0.13021,0.05318,0.89035,0.37073,1.77034,0.01384,...,1.16956,0.76954,0.10651,4.12286,0.25826,0.33129,0.03426,PC+MO,여성,전체
2021-05-11,1.94569,0.06119,0.26127,0.07788,0.12965,0.05365,0.84614,0.31753,1.62360,0.01252,...,1.06354,0.69509,0.10076,3.64909,0.22952,0.31582,0.03125,PC+MO,여성,전체
2021-05-12,1.98369,0.05335,0.24103,0.07855,0.12169,0.04946,0.84232,0.28988,1.77485,0.01175,...,1.03649,0.67466,0.09944,3.48401,0.21674,0.29607,0.03120,PC+MO,여성,전체
2021-05-13,1.85420,0.05351,0.21888,0.07624,0.11261,0.04681,0.76584,0.26450,2.57079,0.01195,...,0.96786,0.70490,0.10464,3.34698,0.20285,0.27130,0.02851,PC+MO,여성,전체
2021-05-14,1.70134,0.05088,0.20506,0.06590,0.10359,0.04320,0.69227,0.24881,3.24689,0.01217,...,0.88763,0.64085,0.08677,3.02990,0.18225,0.23819,0.02345,PC+MO,여성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,1.55276,0.02485,0.10244,0.05113,0.06751,0.02633,0.36933,0.17931,2.79920,0.01220,...,0.56088,0.59611,0.06178,2.21686,0.10646,0.18891,0.03142,MO,여성,전체
2023-06-30,1.33742,0.02165,0.10051,0.04743,0.06392,0.02457,0.36675,0.18453,2.33014,0.01248,...,0.50435,0.54150,0.06533,1.99933,0.10721,0.17183,0.02973,MO,여성,전체
2023-07-01,1.70749,0.02487,0.11087,0.04754,0.06679,0.02765,0.37403,0.23255,2.70753,0.01687,...,0.60500,0.62910,0.07102,2.30425,0.13285,0.20279,0.02881,MO,여성,전체


In [5]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        {"name": "PC", "param": [ "50000089"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "광학기기/용품", "param": [ "50000207"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"m",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["디지털/가전"], axis=1, inplace=True)
df_all_c

,휴대폰액세서리,PC,노트북,PC부품,노트북액세서리,소프트웨어,주변기기,게임기/타이틀,계절가전,광학기기/용품,...,음향가전,이미용가전,자동차기기,주방가전,카메라/캠코더용품,태블릿PC액세서리,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,6.76690,0.74484,2.54605,2.44756,0.53764,0.33873,4.46525,2.95513,6.21716,0.10369,...,6.44718,2.21969,1.95800,8.30939,1.47230,0.88645,0.09665,PC+MO,남성,전체
2021-05-11,6.47648,0.69394,2.24449,2.43001,0.51329,0.34620,4.05452,2.52988,5.52468,0.10782,...,5.93795,2.07992,1.85792,7.13494,1.21220,0.84468,0.08415,PC+MO,남성,전체
2021-05-12,6.52790,0.66623,2.19362,2.48005,0.51081,0.32997,3.90761,2.36460,6.06290,0.10234,...,5.79047,2.00519,1.74547,6.94758,1.20381,0.82980,0.08158,PC+MO,남성,전체
2021-05-13,5.99403,0.60079,1.92416,2.22119,0.47351,0.29935,3.52576,2.21163,8.34568,0.09876,...,5.41476,1.97071,1.67509,6.67940,1.04576,0.72322,0.07343,PC+MO,남성,전체
2021-05-14,5.51944,0.56367,1.81403,2.03312,0.43088,0.26165,3.22989,2.10337,9.94786,0.08954,...,5.12732,1.86628,1.54723,6.14338,1.01330,0.64299,0.07316,PC+MO,남성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,6.27386,0.32341,1.06947,1.60017,0.35449,0.15947,2.39490,1.98028,9.14756,0.11820,...,3.96551,1.06210,1.42143,5.24536,0.80518,0.66444,0.10712,MO,남성,전체
2023-06-30,5.66976,0.31935,1.03225,1.48673,0.32660,0.16163,2.25160,1.88476,7.93140,0.09756,...,3.71628,1.00284,1.35767,4.76300,0.75041,0.61923,0.09466,MO,남성,전체
2023-07-01,6.65925,0.34192,1.12104,1.45568,0.34821,0.16207,2.30574,2.34013,9.81182,0.11760,...,4.35552,1.22348,1.40347,5.84909,0.93539,0.71300,0.11229,MO,남성,전체


In [6]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7065 entries, 2021-05-10 to 2023-07-03
Data columns (total 23 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   휴대폰액세서리    7065 non-null   float64
 1   PC         7065 non-null   float64
 2   노트북        7065 non-null   float64
 3   PC부품       7065 non-null   float64
 4   노트북액세서리    7065 non-null   float64
 5   소프트웨어      7065 non-null   float64
 6   주변기기       7065 non-null   float64
 7   게임기/타이틀    7065 non-null   float64
 8   계절가전       7065 non-null   float64
 9   광학기기/용품    7065 non-null   float64
 10  모니터        7065 non-null   float64
 11  생활가전       7065 non-null   float64
 12  영상가전       7065 non-null   float64
 13  음향가전       7065 non-null   float64
 14  이미용가전      7065 non-null   float64
 15  자동차기기      7065 non-null   float64
 16  주방가전       7065 non-null   float64
 17  카메라/캠코더용품  7065 non-null   float64
 18  태블릿PC액세서리  7065 non-null   float64
 19  학습기기       7065 non-null   float64
 20

In [7]:
## 엑셀에 저장
df_all.to_excel("MCLS_04_all_fm_all.xlsx", sheet_name="click ratio")
df_all.head()

,휴대폰액세서리,PC,노트북,PC부품,노트북액세서리,소프트웨어,주변기기,게임기/타이틀,계절가전,광학기기/용품,...,음향가전,이미용가전,자동차기기,주방가전,카메라/캠코더용품,태블릿PC액세서리,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,1.43483,0.09399,0.30502,0.32940,0.09436,0.04875,0.82239,0.32281,1.11915,0.01257,...,1.02346,0.38094,0.20504,1.84242,0.17984,0.33816,0.01649,PC+MO,전체,전체
2021-05-11,1.37692,0.08884,0.27591,0.31580,0.09136,0.04819,0.77576,0.27776,1.10075,0.01224,...,0.94563,0.35392,0.19436,1.64319,0.15274,0.33506,0.01514,PC+MO,전체,전체
2021-05-12,1.41037,0.08398,0.26763,0.31932,0.08920,0.04674,0.76831,0.25925,1.16766,0.01142,...,0.94872,0.34595,0.18748,1.60637,0.14968,0.32561,0.01485,PC+MO,전체,전체
2021-05-13,1.31968,0.07725,0.23890,0.30471,0.08314,0.04337,0.71071,0.24180,1.55491,0.01153,...,0.90633,0.34923,0.18634,1.55192,0.13434,0.30578,0.01349,PC+MO,전체,전체
2021-05-14,1.24008,0.07210,0.23315,0.28996,0.08032,0.03804,0.64772,0.23084,1.85131,0.01067,...,0.88298,0.33206,0.17345,1.42553,0.12583,0.29567,0.01213,PC+MO,전체,전체
